In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
from PIL import Image

In [4]:
import numpy as np

In [5]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19,self).__init__()
        self.conv1_1=nn.Conv2d(3,64,kernel_size=3)
        self.conv1_2=nn.Conv2d(64,64,kernel_size=3)

        self.conv2_1=nn.Conv2d(64,128,kernel_size=3)
        self.conv2_2=nn.Conv2d(128,128,kernel_size=3)

        self.conv3_1=nn.Conv2d(128,256,kernel_size=3)
        self.conv3_2=nn.Conv2d(256,256,kernel_size=3)
        self.conv3_3=nn.Conv2d(256,256,kernel_size=3)
        self.conv3_4=nn.Conv2d(256,256,kernel_size=3)

        self.conv4_1=nn.Conv2d(256,512,kernel_size=3)
        self.conv4_2=nn.Conv2d(512,512,kernel_size=3)
        self.conv4_3=nn.Conv2d(512,512,kernel_size=3)
        self.conv4_4=nn.Conv2d(512,512,kernel_size=3)

        self.conv5_1=nn.Conv2d(256,512,kernel_size=3)
        self.conv5_2=nn.Conv2d(512,512,kernel_size=3)
        self.conv5_3=nn.Conv2d(512,512,kernel_size=3)
        self.conv5_4=nn.Conv2d(512,512,kernel_size=3)

        self.conv5_drop=nn.Dropout2d()

        self.fc1=nn.Linear(25088,4096)
        self.fc2=nn.Linear(4096,4096)
        self.fc3=nn.Linear(4096,1000)
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1_2(self.conv1_1(x)),2))
        x=F.relu(F.max_pool2d(self.conv2_2(self.conv2_1(x)),2))
        x=F.relu(F.max_pool2d(self.conv3_4(self.conv3_3(self.conv3_2(self.conv3_1(x)))),2))
        x=F.relu(F.max_pool2d(self.conv4_4(self.conv4_3(self.conv4_2(self.conv4_1(x)))),2))
        x=F.relu(F.max_pool2d(self.conv5_drop(self.conv5_3(self.conv5_4(self.conv5_2(self.conv5_1(x))))),2)) 
        x=x.view(-1,25088)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc2(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc3(x))
        return F.softmax(x,dim=1)

In [11]:
model=VGG19()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.005,momentum=0.9)

In [12]:
cuda_gpu=torch.cuda.is_available()

In [13]:
def train(model,criterion,optimizer,dataset):
    model.train()
    for data,target in dataset:
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        optimizer.zero_grad()
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
    

In [15]:
def test(model,criterion,data_loader):
    model.eval()
    test_loss=0
    correct=0
    for data,target in data_loader:
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        test_loss+=criterion(output,target).data[0]
        pred=output.data.max(1)[1]
        correct+=pred.eq(target.data).cpu().sum()
    test_loss/=len(data_loader)
    acc=correct/len(data_loader)
    return(acc,test_loss)